In [1]:
import pandas as pd
import base64
import numpy as np
import cv2
from skimage import transform
import io
from PIL import Image
import os

pd.options.display.max_colwidth=100

In [2]:
PATH = '/home/msnow/msceleb/'

In [3]:
df_names_en = pd.read_feather(f'{PATH}df_names_feather')
df_names_en.head()

,MID,name,short
0,m.01008l47,Patrick Cummins@en,Patrick_Cummins
1,m.01008l96,Mohamed Guessous@en,Mohamed_Guessous
2,m.01008lnk,Tsvetta Kaleynska@en,Tsvetta_Kaleynska
3,m.01008lp2,Caio Henrique Siqueira Sanchez@en,Caio_Henrique_Siqueira_Sanchez
4,m.01008nqj,Julio Ríos Gallego@en,Julio_Ríos_Gallego


In [3]:
# df_names = pd.read_table(f'{PATH}Top1M_MidList.Name.tsv', names=['MID','name'], delimiter='\t')
# df_names_en = df_names.loc[df_names.name.str.contains('@en$'),:].reset_index(drop=True)
# df_names_en['short'] = df_names_en.name.str.replace('@en','')
# df_names_en.short = df_names_en.short.str.replace(' ','_')
# # df_names_en.to_feather(f'{PATH}df_names_feather')
# df_names_en.head()

,MID,name
0,m.01008l47,Patrick Cummins@en
1,m.01008l96,Mohamed Guessous@en
2,m.01008lnk,Tsvetta Kaleynska@en
3,m.01008lp2,Caio Henrique Siqueira Sanchez@en
4,m.01008nqj,Julio Ríos Gallego@en


In [4]:
df = pd.read_csv(f'{PATH}df_images.csv')
# df = pd.read_table(f'{PATH}FaceImageCroppedWithOutAlignment.tsv', nrows=1058604, delimiter='\t', names=['MID','rank','url','page_url','faceid','FaceRectangle','face_data'])
# df.drop(columns=['rank','page_url','FaceRectangle'], inplace=True)
# # df.to_csv(f'{PATH}df_images.csv',index=False)
df.head()

,MID,url,faceid,face_data
0,m.0107_f,http://getbeatmadrid.files.wordpress.com/2013/01/magic-alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
1,m.0107_f,http://1.bp.blogspot.com/-HNFIL7eKdNs/TxnlvZvisvI/AAAAAAAAANE/AsNempI4Efc/s1600/magicalex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
2,m.0107_f,http://www.magicalex.cz/wp-content/gallery/kouzla/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
3,m.0107_f,http://2.bp.blogspot.com/_71dhc6LM1i4/SBSPDqHtjPI/AAAAAAAAAVg/rFWbkmqfo2Q/s200/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
4,m.0107_f,http://www.magicalex.cz/wp-content/uploads/2013/02/Magic-Alex-foto-2013.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...


In [7]:
# df = pd.read_table(f'{PATH}FaceImageCroppedWithOutAlignment.tsv', nrows=1058604, skiprows=0, delimiter='\t', names=['MID','rank','url','page_url','faceid','FaceRectangle','face_data'])
# df.head()

In [5]:
df_merge = df.merge(df_names_en,how='left')
df_merge.head()

,MID,url,faceid,face_data,name,short
0,m.0107_f,http://getbeatmadrid.files.wordpress.com/2013/01/magic-alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex
1,m.0107_f,http://1.bp.blogspot.com/-HNFIL7eKdNs/TxnlvZvisvI/AAAAAAAAANE/AsNempI4Efc/s1600/magicalex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex
2,m.0107_f,http://www.magicalex.cz/wp-content/gallery/kouzla/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex
3,m.0107_f,http://2.bp.blogspot.com/_71dhc6LM1i4/SBSPDqHtjPI/AAAAAAAAAVg/rFWbkmqfo2Q/s200/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex
4,m.0107_f,http://www.magicalex.cz/wp-content/uploads/2013/02/Magic-Alex-foto-2013.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex


remove all photos with more than one face in them

In [6]:
multi_face = df_merge.loc[df_merge.faceid!='FaceId-0','url']
multi_face = multi_face.unique().tolist()
df_merge_one = df_merge.loc[~df_merge.url.isin(multi_face),:]
df_merge_one.shape[0]/df_merge.shape[0]

0.8597464207579038

In [8]:
df_merge_one['name_ct'] = df_merge_one.groupby('MID').cumcount()
df_merge_one['save_nm'] = df_merge_one.short + '-' + df_merge_one.name_ct.astype(str).str.zfill(4)
df_merge_one.head()

/home/msnow/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/msnow/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,MID,url,faceid,face_data,name,short,name_ct,save_nm
0,m.0107_f,http://getbeatmadrid.files.wordpress.com/2013/01/magic-alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex,0,Magic_Alex-0000
1,m.0107_f,http://1.bp.blogspot.com/-HNFIL7eKdNs/TxnlvZvisvI/AAAAAAAAANE/AsNempI4Efc/s1600/magicalex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex,1,Magic_Alex-0001
2,m.0107_f,http://www.magicalex.cz/wp-content/gallery/kouzla/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex,2,Magic_Alex-0002
3,m.0107_f,http://2.bp.blogspot.com/_71dhc6LM1i4/SBSPDqHtjPI/AAAAAAAAAVg/rFWbkmqfo2Q/s200/magic_alex.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex,3,Magic_Alex-0003
4,m.0107_f,http://www.magicalex.cz/wp-content/uploads/2013/02/Magic-Alex-foto-2013.jpg,FaceId-0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...,Magic Alex@en,Magic_Alex,4,Magic_Alex-0004


In [9]:
many_ct = df_merge_one.loc[df_merge_one.name_ct==30,'MID'].tolist()
df_many = df_merge_one.loc[df_merge_one.MID.isin(many_ct),:]
df_many.shape

(890870, 8)

In [29]:
for i,v in df_many.iterrows():
    savePath = os.path.join(PATH,'images',f'{v.save_nm}.jpg')
    with open(savePath, 'wb') as f:
        f.write(base64.b64decode(v.face_data))  

In [12]:
df_save = df_many.loc[:,['MID','url','short','name_ct','save_nm']]
df_save.to_csv(f'{PATH}msceleb_img_ref.csv',index=False)

In [37]:
curr_name =''
name_ct = 0
for i,v in df_merge_one.iterrows():
    if v.short != curr_name:
        name_ct = 0
        curr_name = v.short
    else:
        name_ct += 1
    name_ct_str = str(name_ct).zfill(3)
    savePath = os.path.join(PATH,'images',f'{v.short}_{name_ct_str}.jpg')
    with open(savePath, 'wb') as f:
        f.write(base64.b64decode(v.face_data))   

FileNotFoundError: [Errno 2] No such file or directory: '/home/msnow/msceleb/images/Joint_POW/MIA_Accounting_Command_000.jpg'

In [64]:
row_num=1000
row = df.iloc[row_num,:]
MID, imgSearchRank, faceID, data = row[0], row[1], row[4], base64.b64decode(row[6])
savePath = "{}-{}.jpg".format(imgSearchRank, faceID)
with open(savePath, 'wb') as f:
    f.write(data)
savePath

'6-FaceId-0.jpg'

<img src = '6-FaceId-0.jpg'>

# Correct for name counting at the beginning and end of every file

In [75]:
df_short = pd.read_table(f'{PATH}FaceImageCroppedWithOutAlignment.tsv', nrows=2*10**3, skiprows=(10**6 - 10**3), delimiter='\t', names=['MID','rank','url','page_url','faceid','FaceRectangle','face_data'])
df_short.head()

,MID,rank,url,page_url,faceid,FaceRectangle,face_data
0,m.025xdd6,33,http://celebrityink.eu/image/cache/data/3847F-1000x1000.jpg,http://filmvz.com/nude/nude-celebrity-videos-of-beatie-edney-in-highlander-rainpow.htm,FaceId-0,TDeJPpLt/D4yCAw/VeNFPw==,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
1,m.025xdd6,34,http://i11.photobucket.com/albums/a176/Skylover_MadCollectorZ/for%20trade-sell/non-star%20wars/1...,http://photobucket.com/images/Beatie%20Edney%20-%20Heather%20MacLeod%20(Duncan%20McLeod%20wife%2...,FaceId-0,RUTEPqNwfT61gS4/rkfxPg==,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
2,m.025xdd6,35,http://cdn.cinemur.fr/persons/media/original/beatie-edney_15297_5220dc4cdd5ec.jpg,http://cinemur.fr/personnes/beatie-edney-15297,FaceId-0,AICcPloFxj4AABQ/SHFfPw==,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
3,m.025xdd6,36,http://media.mademan.com/chickipedia/uploaded_photos/6/6a/Beatie_Edney-stunning-before-soft-skin...,http://filmvz.com/nude/nude-celebs-beatie-edney-english-celebrities-emma-bunton-gets.htm,FaceId-0,gQqoPj40lj5hCzY/sT4UPw==,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...
4,m.025xdd6,37,http://media.mademan.com/chickipedia/uploaded_photos/5/5c/Beatie_Edney-young-babe-skin-eyes-chic...,http://www.mademan.com/chickipedia/beatie-edney/photosgallery/Beatie_Edney-young-babe-skin-eyes-...,FaceId-0,08+cPg68qD648yI/7gEOPw==,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAwMEBAQEBQkGBQUFBQsICAYJDQsNDQ0LDAwOEBQRDg8TDwwMEhgS...


Remove first and last person for list

In [76]:
MID_ignore = [df_short.MID[0], df_short.MID[df_short.shape[0]-1]]
df_short = df_short.loc[~df_short.MID.isin(MID_ignore),:]
df_short.shape

(1911, 7)

In [77]:
df_merge = df_short.merge(df_names_en,how='left')
multi_face = df_merge.loc[df_merge.faceid!='FaceId-0','url']
multi_face = multi_face.unique().tolist()
df_merge_one = df_merge.loc[~df_merge.url.isin(multi_face),:]
df_merge_one.shape[0]/df_merge.shape[0]

0.7760334903192047

In [78]:
curr_name =''
name_ct = 0
for i,v in df_merge_one.iterrows():
    if v.short != curr_name:
        name_ct = 0
        curr_name = v.short
    else:
        name_ct += 1
    name_ct_str = str(name_ct).zfill(3)
    savePath = os.path.join(PATH,'images',f'{v.short}_{name_ct_str}.jpg')
    with open(savePath, 'wb') as f:
        f.write(base64.b64decode(v.face_data)) 

# Create a list of all the names

In [ ]:
1058604

In [10]:
df_nrw2 = pd.read_table(f'{PATH}FaceImageCroppedWithOutAlignment.tsv', nrows=(5*10**5+58604), skiprows=5*10**5, delimiter='\t', names=['MID','url','faceid'],usecols=[0,2,4])
df_nrw2.head()

,MID,url,faceid
0,m.01ptrmd,http://4.bp.blogspot.com/-XxC7Cwnk9r0/U3eMGs7ulAI/AAAAAAAA-2g/DzdjAdzjQAU/s1600/Vico+C+-+Histori...,FaceId-0
1,m.01ptrmd,http://www.caratulas.com/caratulas/V/vico_c/vico_c-greatest_hits-Interior_Frontal.jpg,FaceId-0
2,m.01ptrmd,http://www.caratulas.com/caratulas/V/vico_c/vico_c-greatest_hits-Interior_Frontal.jpg,FaceId-1
3,m.01ptrmd,http://i.ytimg.com/vi/eo5SqSPlFIE/hqdefault.jpg,FaceId-0
4,m.01ptrmd,http://s1.evcdn.com/images/edpborder500/I0-001/001/721/212-5.jpeg_/vico-c-12.jpeg,FaceId-0


In [26]:
df_merge2 = df_nrw2.merge(df_names_en,how='left')
df_merge2.head()

,MID,url,faceid,name,short
0,m.01ptrmd,http://4.bp.blogspot.com/-XxC7Cwnk9r0/U3eMGs7ulAI/AAAAAAAA-2g/DzdjAdzjQAU/s1600/Vico+C+-+Histori...,FaceId-0,Vico C@en,Vico_C
1,m.01ptrmd,http://www.caratulas.com/caratulas/V/vico_c/vico_c-greatest_hits-Interior_Frontal.jpg,FaceId-0,Vico C@en,Vico_C
2,m.01ptrmd,http://www.caratulas.com/caratulas/V/vico_c/vico_c-greatest_hits-Interior_Frontal.jpg,FaceId-1,Vico C@en,Vico_C
3,m.01ptrmd,http://i.ytimg.com/vi/eo5SqSPlFIE/hqdefault.jpg,FaceId-0,Vico C@en,Vico_C
4,m.01ptrmd,http://s1.evcdn.com/images/edpborder500/I0-001/001/721/212-5.jpeg_/vico-c-12.jpeg,FaceId-0,Vico C@en,Vico_C


In [27]:
multi_face = df_merge2.loc[df_merge2.faceid!='FaceId-0','url']
multi_face = multi_face.unique().tolist()
df_merge_one2 = df_merge2.loc[~df_merge2.url.isin(multi_face),:]
df_merge_one2.shape[0]/df_merge2.shape[0]

0.8447104745787575

In [28]:
df_merge_one2.shape

(472194, 5)

In [34]:
nm_ct_vct2 = nm_ct_vct

In [39]:
nm_ct = -1
cur_nm = ''
# nm_ct_vct = []
for i,v in df_merge_one2.iterrows():
    if v.short == cur_nm:
        nm_ct += 1
    else:
        cur_nm = v.short
        nm_ct = 0
    nm_ct_vct.append(nm_ct)

In [35]:
df_merge_one.shape[0] + df_merge_one2.shape[0]

910946

In [40]:
len(nm_ct_vct)

910946

In [24]:
len(nm_ct_vct)

431644

In [41]:
df_merge_comb = df_merge_one.append(df_merge_one2, ignore_index=True)
df_merge_comb['name_ct'] = nm_ct_vct
df_merge_comb.shape

(910946, 6)

In [43]:
df_merge_comb.drop(columns=['faceid','name'],inplace=True)
df_merge_comb.head()

,MID,url,short,name_ct
0,m.0107_f,http://getbeatmadrid.files.wordpress.com/2013/01/magic-alex.jpg,Magic_Alex,0
1,m.0107_f,http://1.bp.blogspot.com/-HNFIL7eKdNs/TxnlvZvisvI/AAAAAAAAANE/AsNempI4Efc/s1600/magicalex.jpg,Magic_Alex,1
2,m.0107_f,http://www.magicalex.cz/wp-content/gallery/kouzla/magic_alex.jpg,Magic_Alex,2
3,m.0107_f,http://2.bp.blogspot.com/_71dhc6LM1i4/SBSPDqHtjPI/AAAAAAAAAVg/rFWbkmqfo2Q/s200/magic_alex.jpg,Magic_Alex,3
4,m.0107_f,http://www.magicalex.cz/wp-content/uploads/2013/02/Magic-Alex-foto-2013.jpg,Magic_Alex,4


In [82]:
df_merge_comb['name_ct'] = df_merge_comb.groupby('MID').cumcount()

In [84]:
df_merge_comb.to_csv(f'{PATH}img_files.csv',index=False)

In [88]:
(df_merge_comb.short=='Scott_Baio-GB').sum()

0